# E₈ Root System 3D Visualization

Interactive 3D visualization of the complete E₈ root system (240 roots).

The E₈ Lie algebra is central to the GIFT framework, providing the foundational structure for dimensional reduction. This visualization shows the complete root system projected from 8D to 3D using Principal Component Analysis.

**Mathematical background**: See Supplement A (Mathematical Foundations) for detailed E₈ structure.

**Key properties**:
- Rank: 8
- Dimension: 248 (8 Cartan + 240 roots)
- Root system: 240 vectors in 8D space
- Two root lengths: 120 short roots + 120 long roots
- Weyl group order: 696,729,600 = 2¹⁴ × 3⁵ × 5² × 7


In [ ]:
import numpy as np
import plotly.graph_objects as go
from sklearn.decomposition import PCA
import pandas as pd

print("E₈ Root System Visualization")
print("="*50)


## 1. Generate E₈ Root System

We construct the 240 roots using the standard E₈ root system in ℝ⁸.

**Construction method**: Use the explicit coordinates where E₈ roots are:
1. All permutations of (±1, ±1, 0, 0, 0, 0, 0, 0) [112 roots]
2. All vectors (½, ½, ½, ½, ½, ½, ½, ½) with even number of minus signs [128 roots]

Total: 240 roots


In [ ]:
def generate_e8_roots():
    """
    Generate all 240 roots of E₈ Lie algebra.
    
    Returns:
        np.ndarray: (240, 8) array of root vectors
        list: Root types for coloring
    """
    roots = []
    root_types = []
    
    # Type 1: All permutations of (±1, ±1, 0, 0, 0, 0, 0, 0)
    # Choose 2 positions out of 8 for the non-zero entries: C(8,2) = 28
    # Each position can be ±1: 2² = 4 choices
    # Total: 28 × 4 = 112 roots
    from itertools import combinations, product
    
    for positions in combinations(range(8), 2):
        for signs in product([1, -1], repeat=2):
            root = np.zeros(8)
            root[positions[0]] = signs[0]
            root[positions[1]] = signs[1]
            roots.append(root)
            root_types.append('Type 1')
    
    # Type 2: (½, ½, ½, ½, ½, ½, ½, ½) with even number of minus signs
    # 2⁸ = 256 total sign combinations, 128 with even parity
    for signs in product([1, -1], repeat=8):
        if signs.count(-1) % 2 == 0:  # Even number of minus signs
            root = np.array([s * 0.5 for s in signs])
            roots.append(root)
            root_types.append('Type 2')
    
    roots = np.array(roots)
    print(f"Generated {len(roots)} E₈ roots")
    print(f"  Type 1 (permutations of ±1,±1,0,...): {root_types.count('Type 1')} roots")
    print(f"  Type 2 (½ vectors, even parity): {root_types.count('Type 2')} roots")
    
    # Calculate root lengths
    lengths = np.linalg.norm(roots, axis=1)
    print(f"\nRoot length statistics:")
    print(f"  Short roots (length √2): {np.sum(np.abs(lengths - np.sqrt(2)) < 0.01)}")
    print(f"  Long roots (length 2): {np.sum(np.abs(lengths - 2.0) < 0.01)}")
    
    return roots, root_types

# Generate roots
e8_roots, root_types = generate_e8_roots()


## 2. Project to 3D using PCA

Principal Component Analysis projects the 8D root system to 3D while preserving maximum variance.


In [ ]:
# Project from 8D to 3D using PCA
pca = PCA(n_components=3)
roots_3d = pca.fit_transform(e8_roots)

explained_variance = pca.explained_variance_ratio_
print(f"\nPCA Projection to 3D:")
print(f"  PC1 variance: {explained_variance[0]:.3f}")
print(f"  PC2 variance: {explained_variance[1]:.3f}")
print(f"  PC3 variance: {explained_variance[2]:.3f}")
print(f"  Total variance preserved: {explained_variance.sum():.3f}")

# Calculate root lengths for coloring
lengths = np.linalg.norm(e8_roots, axis=1)
is_short = np.abs(lengths - np.sqrt(2)) < 0.01
is_long = np.abs(lengths - 2.0) < 0.01


## 3. Interactive 3D Visualization

Explore the E₈ root system structure with interactive controls:
- **Rotate**: Click and drag
- **Zoom**: Scroll or pinch
- **Pan**: Right-click and drag
- **Colors**: Red = short roots (√2), Blue = long roots (2)


In [ ]:
# Create interactive 3D scatter plot
fig = go.Figure()

# Short roots (red)
fig.add_trace(go.Scatter3d(
    x=roots_3d[is_short, 0],
    y=roots_3d[is_short, 1],
    z=roots_3d[is_short, 2],
    mode='markers',
    name='Short roots (√2)',
    marker=dict(
        size=6,
        color='red',
        opacity=0.8,
        line=dict(width=0.5, color='darkred')
    ),
    hovertemplate='Short root<br>Length: √2<br>PC1: %{x:.3f}<br>PC2: %{y:.3f}<br>PC3: %{z:.3f}<extra></extra>'
))

# Long roots (blue)
fig.add_trace(go.Scatter3d(
    x=roots_3d[is_long, 0],
    y=roots_3d[is_long, 1],
    z=roots_3d[is_long, 2],
    mode='markers',
    name='Long roots (2)',
    marker=dict(
        size=6,
        color='blue',
        opacity=0.8,
        line=dict(width=0.5, color='darkblue')
    ),
    hovertemplate='Long root<br>Length: 2<br>PC1: %{x:.3f}<br>PC2: %{y:.3f}<br>PC3: %{z:.3f}<extra></extra>'
))

# Add origin
fig.add_trace(go.Scatter3d(
    x=[0], y=[0], z=[0],
    mode='markers',
    name='Origin',
    marker=dict(size=10, color='black', symbol='diamond'),
    hovertemplate='Origin<extra></extra>'
))

# Layout
fig.update_layout(
    title=dict(
        text='E₈ Root System - 240 Roots in 3D Projection',
        x=0.5,
        xanchor='center',
        font=dict(size=20)
    ),
    scene=dict(
        xaxis_title='PC1 ({:.1f}% variance)'.format(explained_variance[0]*100),
        yaxis_title='PC2 ({:.1f}% variance)'.format(explained_variance[1]*100),
        zaxis_title='PC3 ({:.1f}% variance)'.format(explained_variance[2]*100),
        camera=dict(
            eye=dict(x=1.5, y=1.5, z=1.5)
        ),
        aspectmode='cube'
    ),
    width=1000,
    height=800,
    showlegend=True,
    legend=dict(
        x=0.02,
        y=0.98,
        bgcolor='rgba(255,255,255,0.8)'
    )
)

fig.show()

print("\nVisualization complete!")
print(f"Total roots displayed: {len(roots_3d)}")


## 4. Statistical Analysis

Analyze the geometric properties of the E₈ root system.


In [ ]:
# Root statistics
print("E₈ Root System Statistics")
print("="*50)
print(f"Total roots: {len(e8_roots)}")
print(f"Dimension: 8D (projected to 3D)")
print(f"\nRoot distribution:")
print(f"  Short roots (length √2 ≈ 1.414): {is_short.sum()}")
print(f"  Long roots (length 2.000): {is_long.sum()}")

# Verify root inner products
inner_products = e8_roots @ e8_roots.T
unique_inner_products = np.unique(np.round(inner_products, 6))
print(f"\nInner product structure:")
print(f"  Unique inner product values: {len(unique_inner_products)}")
print(f"  Values: {unique_inner_products[:10]}...")

# GIFT framework connections
print("\n" + "="*50)
print("GIFT Framework Connections")
print("="*50)
print(f"rank(E₈) = 8")
print(f"dim(E₈) = 248 = 8 (Cartan) + 240 (roots)")
print(f"\nWeyl group W(E₈) order: 696,729,600")
print(f"  = 2¹⁴ × 3⁵ × 5² × 7")
print(f"  Weyl factor in GIFT: 5")
print(f"\nKey relation: N_gen = rank(E₈) - Weyl = 8 - 5 = 3")
print(f"  This explains three generations of fermions!")


## 5. Export Options

Save the visualization for documentation.


In [ ]:
# Save as interactive HTML
output_file = 'e8_root_system_3d.html'
fig.write_html(output_file)
print(f"\nInteractive visualization saved to: {output_file}")
print("Open this file in a web browser for full interactivity.")

# Optional: Save static image (requires kaleido)
try:
    fig.write_image('e8_root_system_3d.png', width=1200, height=900)
    print("Static image saved to: e8_root_system_3d.png")
except Exception as e:
    print(f"Note: Static image export not available (install kaleido): {e}")


## References

- **Supplement A**: Mathematical Foundations - E₈ Lie algebra structure
- **Main Paper**: Section on E₈×E₈ gauge structure
- **Construction**: Standard E₈ root system in 8-dimensional space

## Notes

The E₈ root system exhibits remarkable symmetry and is the largest exceptional simple Lie algebra. In the GIFT framework, two copies of E₈ (E₈×E₈) provide the fundamental 496-dimensional information architecture that reduces to the 99-dimensional cohomology structure of K₇.
